In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import numpy as np
from tqdm import tqdm

In [2]:
driver = webdriver.Firefox()
url = "https://myneta.info/"
driver.get(url)
driver.maximize_window()

In [3]:
state_assemblies = driver.find_elements(By.CLASS_NAME, "w3-card")[7]

In [4]:
assembly_links = state_assemblies.find_elements(By.CLASS_NAME, "w3-button")

In [5]:
assembly_links = [link.get_attribute("href") for link in assembly_links]

In [6]:

winners_links = []

for link in tqdm(assembly_links):
        driver.get(link)
        sleep(1)
        latest_election = driver.find_element(By.CLASS_NAME, "w3-ul")
        latest_election.find_element(By.CLASS_NAME, "w3-block").click()
        sleep(1)
        winners_table = driver.find_element(By.CLASS_NAME, "w3-responsive").find_element(By.CLASS_NAME, "w3-table")
        winners_rows = winners_table.find_elements(By.TAG_NAME, "tr")[1:]
        links = [[row.find_elements(By.TAG_NAME, "a")[1].text, row.find_elements(By.TAG_NAME, "a")[1].get_attribute("href")] for row in winners_rows]
        winners_links += links

100%|██████████| 31/31 [03:21<00:00,  6.50s/it]


In [9]:
winners_links = np.array(winners_links)

In [23]:
import pandas as pd

def get_stateyear(string):
        return string.split("/")[-2]

def get_id(string):
        return string.split("=")[-1]

winners = pd.DataFrame(winners_links, columns=["name", "link"])
winners["state_year"] = winners["link"].apply(get_stateyear)
winners["unique_id"] = winners["state_year"] + "_" + winners["link"].apply(get_id)

In [25]:
winners.to_csv("../data/winners.csv")